In [139]:
%useLatestDescriptors
%use lets-plot

In [140]:
import org.litote.kmongo.serialization.SerializationClassMappingTypeService
import com.mongodb.client.MongoClients
import org.litote.kmongo.withKMongo
import java.io.File
import java.util.*

System.setProperty(
    "org.litote.mongo.mapping.service",
    SerializationClassMappingTypeService::class.qualifiedName!!
)

val properties = Properties()
properties.load(
    File("G:\\Organizations\\Duels\\duels-datascience\\dfl.properties").reader()
)

val mongoConnectionUri = properties.getProperty("mongo.clients.uri")
val mongo = MongoClients.create(mongoConnectionUri)

val database = mongo.getDatabase("duels_anticheat").withKMongo()


In [141]:
import kotlinx.serialization.Contextual
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import org.bson.types.ObjectId
import org.litote.kmongo.*
import java.text.SimpleDateFormat
import java.time.*
import kotlin.collections.count

@Serializable
data class AnticheatLog(
    @SerialName("_id")
    val _id: @Contextual ObjectId,
    val timestamp: Long,
    val ping: Long,
    val vl: Int,
    val check: String
)

val startOfToday = LocalDate
    .now(ZoneId.systemDefault())
    .atTime(LocalTime.MIDNIGHT)
    .atZone(ZoneId.systemDefault())
    
val formatter = SimpleDateFormat("hh.mm a")
    
val logs = database.getCollection("logs").withKMongo()
val result = logs.aggregate<AnticheatLog>(
    match(
        AnticheatLog::check eq "Double Click A",
        AnticheatLog::timestamp gte startOfToday.toInstant().toEpochMilli()
    )
).toList()

val earliestTimestamp = result.minOfOrNull { it.timestamp } ?: Instant.now().toEpochMilli()
val latestTimestamp = result.maxOfOrNull { it.timestamp } ?: Instant.now().toEpochMilli()

val intervalDuration = Duration.ofMinutes(5).seconds * 1000

val bins = ((earliestTimestamp / intervalDuration * intervalDuration)..(latestTimestamp / intervalDuration * intervalDuration) step intervalDuration)

val violationsCountInInterval = bins.map { binStart ->
    val binEnd = binStart + intervalDuration
    val violationsInBin = result.count { log ->
        log.timestamp in binStart until binEnd
    }
    
    binStart to violationsInBin
}

val (timeStamps, violationCounts) = violationsCountInInterval.unzip()
val plotData = mapOf<String, Any>(
    "Time (5 minute intervals)" to timeStamps.map { formatter.format(Date(it)) },
    "Total Double Clicking Violations" to violationCounts
)

val plot = letsPlot(plotData) +
        geomBar(stat = Stat.identity) {
            x = "Time (5 minute intervals)"
            y = "Total Double Clicking Violations"
        }

plot